# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.core 
print(azureml.core.VERSION)

1.20.0


In [2]:
from azureml.core.environment import Environment
Environment(name="vaenv")
myenv = Environment(name="vaenv")

In [3]:
from azureml.core.conda_dependencies import CondaDependencies

dependencies = CondaDependencies()
dependencies.add_pip_package("scikit-learn")
dependencies.add_pip_package("numpy==1.15.4")
myenv.python.conda_dependencies=dependencies

In [4]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name='Toria_workspace',
            subscription_id='784d8740-4c70-4d20-8f45-b2bfecf94029',
            resource_group='myresourcegroup',)
exp = Experiment(workspace=ws, name="Toria_workspace")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: Toria_workspace
Azure region: eastus2
Subscription id: 784d8740-4c70-4d20-8f45-b2bfecf94029
Resource group: myresourcegroup


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
compute_target_name = "vpacompute"
compute_cluster = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", max_nodes = 4)
my_target =  ComputeTarget.create(ws, compute_target_name, compute_cluster)
### YOUR CODE HERE ###

In [6]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Overview
## 
This dataset contains information of a growing customer base of a bank. With majority of it's customers being depositors with 
varying size of deposits. In this project, I will be using Automated ML with a classification task to predict the likelihood 
of a liability customer buying personal loans

TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
dat = "https://raw.githubusercontent.com/vikkyfama/Project_Capstone/toribranch/Personal_loans.csv"
ds = TabularDatasetFactory.from_delimited_files(dat)
print(ds)

TabularDataset
{
  "source": [
    "https://raw.githubusercontent.com/vikkyfama/Project_Capstone/toribranch/Personal_loans.csv"
  ],
  "definition": [
    "GetFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}


In [8]:
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn import datasets
from azureml.core.dataset import Dataset
import numpy as np
from traintest2 import clean_data
#from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn import preprocessing


x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y)


Average Experience 20.1046
Average Experience 20.1046


In [9]:
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn import datasets
from azureml.core.dataset import Dataset
import numpy as np
#from train import clean_data
#from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn import preprocessing
import os


#x_train, x_test, y_train, y_test = train_test_split(P_loan, test_size = 0.3, random_state = 0)
#x_train, x_test, y_train, y_test
training_data = pd.concat([x_train, y_train], axis = 1)

if not os.path.isdir('dataset'):
    os.mkdir('dataset')


training_data.to_csv('dataset/loan.csv')
ds = ws.get_default_datastore()
ds.upload(src_dir='./dataset', target_path='loandataset', overwrite=True, show_progress=True)

training_data = Dataset.Tabular.from_delimited_files(path=ds.path('loandataset/loan.csv'))
training_data

Uploading an estimated of 1 files
Uploading ./dataset/loan.csv
Uploaded ./dataset/loan.csv, 1 files out of an estimated total of 1
Uploaded 1 files


{
  "source": [
    "('workspaceblobstore', 'loandataset/loan.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

In [10]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'People_loan_experiment'

experiment=Experiment(ws, experiment_name)

## AutoML settings
For the AutoMLsettings, I specified an experiment_timeout_hours to 0.3 so as to terminate the experiment when this value is reached.
setting enable_early stop to be true whenever the score is not improving in the short term, the iteration_timeout_minutes was set to 5 which is the maximum time for each iteration to run before it terminates. 
Max_concurrent_iterations was set to 4 which was the number of iterations that would be executed in parallel
For the primary_metric I chose the AUC_Weighted because its better at optimizing datasets which are very small and have very large class skew (clasmay not optimize as well for datasets which are very small and have very large class skew (class imbalance). 
Featurization is set to 'auto'so as to allow featurization to be done automatically.

## AutoML Configuration
For the AutoML Configuration, I chose the classification task because i am trying to predict which category an existing customer will fall into based on learnings from its training data.
The training_data is the resulting data from the already split data after it had been converted to a tabular dataset which is what is accepted by autoML.
Label_column_name is the 'Personal loan' which is the column we are trying to predict.
n_cross_validation is set to 5 which is the number of cross validations to perform.

TODO: Explain why you chose the automl settings and cofiguration you used below.


In [11]:
# TODO: Put your automl settings here
automl_settings = {
     "experiment_timeout_hours" : 0.3,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target = 'vpacompute',
    task="classification",
    training_data= training_data,
    label_column_name= "Personal Loan",
    n_cross_validations=5
)

In [12]:
# TODO: Submit your experiment
#remote_run = experiment.submit(automl_config)
remote_run = experiment.submit(automl_config, show_output = True)
remote_run

Running on remote.
No run_configuration provided, running on vpacompute with default configuration
Running on remote compute: vpacompute
Parent Run ID: AutoML_d26f0cc7-9605-402e-9480-92002a6f696a

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Labe

Experiment,Id,Type,Status,Details Page,Docs Page
People_loan_experiment,AutoML_d26f0cc7-9605-402e-9480-92002a6f696a,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [52]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# to install required packages
env = Environment('vfenv')
cd = CondaDependencies.create(pip_packages=['azureml-sdk','azureml-dataset-runtime[pandas,fuse]', 'azureml-defaults'], conda_packages = ['scikit-learn==0.22.1', 'scikit-learn'])
#azureml-sdk", "azureml-dataprep[fuse,pandas]", "azureml-defaults>=1.0.45
env.python.conda_dependencies = cd
#cd.add_conda_package("scikit-learn")
# Register environment to re-use later
env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20210104.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "vfenv",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
      

In [77]:
best_run, best_model = remote_run.get_output()
#print(best_run)
#print(best_model)
#best_run, best_model = remote_run.get_output()
#best_run, best_model
best_run.id



'AutoML_d26f0cc7-9605-402e-9480-92002a6f696a_30'

In [78]:
#TODO: Save the best model
import joblib
#joblib.dump(best_model,'best_aut_model.pkl')



#filename = 'outputs/best_aut_model.joblib'
joblib.dump(best_model, 'best_remoterun_model.pkl')

#print(filename)

#import joblib
#joblib.dump(value=fmodel, filename='model.pkl')

['best_remoterun_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [114]:
from azureml.core.model import Model

#model = best_model.register_model(model_name = 'best_aut_model', model_path = './outputs/')

#from azureml.core.model import Model

model = Model.register(model_path= 'best_remoterun_model.pkl',
                       model_name="best_remoterun_model",
                       tags={"data": "loans",  "method" : "sklearn"},
                       description="Predict loans with sklearn",
                       workspace=ws)
print(model)

Registering model best_remoterun_model
Model(workspace=Workspace.create(name='toria_workspace', subscription_id='784d8740-4c70-4d20-8f45-b2bfecf94029', resource_group='myresourcegroup'), name=best_remoterun_model, id=best_remoterun_model:2, version=2, tags={'data': 'loans', 'method': 'sklearn'}, properties={})


In [119]:
from azureml.core import Workspace, Environment

ws = Workspace.from_config()
env = Environment.get(workspace=ws, name="AzureML-AutoML")



In [129]:
from azureml.automl.core.shared import constants
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'env.yml')

In [89]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment(name="vfenv")
conda_dep = CondaDependencies()

# Installs numpy version 1.17.0 conda package
conda_dep.add_conda_package("numpy==1.18.0")

# Installs pillow package
conda_dep.add_pip_package("pillow")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

In [117]:
envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML-AutoML"):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())

Name AzureML-AutoML
packages channels:
- anaconda
- conda-forge
- pytorch
dependencies:
- python=3.6.2
- pip=20.2.4
- pip:
  - azureml-core==1.20.0
  - azureml-pipeline-core==1.20.0
  - azureml-telemetry==1.20.0
  - azureml-defaults==1.20.0
  - azureml-interpret==1.20.0
  - azureml-automl-core==1.20.0
  - azureml-automl-runtime==1.20.0
  - azureml-train-automl-client==1.20.0
  - azureml-train-automl-runtime==1.20.0.post1
  - azureml-dataset-runtime==1.20.0
  - inference-schema
  - py-cpuinfo==5.0.0
  - boto3==1.15.18
  - botocore==1.18.18
- numpy~=1.18.0
- scikit-learn==0.22.1
- pandas~=0.25.0
- py-xgboost<=0.90
- fbprophet==0.5
- holidays==0.9.11
- setuptools-git
- psutil>5.0.0,<6.0.0
name: azureml_265db83b0c6014ce472c5de2f0b97e04

Name AzureML-AutoML-GPU
packages channels:
- anaconda
- conda-forge
- pytorch
dependencies:
- python=3.6.2
- pip=20.2.4
- pip:
  - azureml-core==1.20.0
  - azureml-pipeline-core==1.20.0
  - azureml-telemetry==1.20.0
  - azureml-defaults==1.20.0
  - azureml-

In [121]:
import sklearn
from azureml.core.conda_dependencies import CondaDependencies

from azureml.core.environment import Environment

#environment = Environment("LocalDeploy")
env.python.conda_dependencies.add_pip_package("scikit-learn==0.22.1".format(sklearn.__version__))

#env.python.conda_dependencies.add_pip_package("sklearn")
#env.python.conda_dependencies.add_pip_package("azureml-defaults")

In [122]:
%%writefile score.py
import json
import logging
import os
import pickle
import numpy as np
import pandas as pd
import joblib
from azureml.core.model import Model
#from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, PublicAccess

import azureml.automl.core
from azureml.automl.core.shared import logging_utilities, log_server
from azureml.telemetry import INSTRUMENTATION_KEY

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType


input_sample = pd.DataFrame({"Column1": pd.Series([0], dtype="int64"), "Experience": pd.Series([0.0], dtype="float64"), "Income": pd.Series([0.0], dtype="float64"), "Family": pd.Series([0], dtype="int64"), "CCAvg": pd.Series([0.0], dtype="float64"), "Education": pd.Series([0], dtype="int64"), "Mortgage": pd.Series([0.0], dtype="float64"), "Securities Account": pd.Series([0], dtype="int64"), "CD Account": pd.Series([0], dtype="int64")})
output_sample = np.array([0])
try:
    log_server.enable_telemetry(INSTRUMENTATION_KEY)
    log_server.set_verbosity('INFO')
    logger = logging.getLogger('azureml.automl.core.scoring_script')
except:
    pass


def init():
    global model
     #This name is model.id of model that we want to deploy deserialize the model file back
    # into a sklearn model
    model_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), 'best_remoterun_model.pkl')
    path = os.path.normpath(model_path)
    path_split = path.split(os.sep)
    log_server.update_custom_dimensions({'model_name': path_split[1], 'model_version': path_split[2]})
    try:
        logger.info("Loading model from path.")
        model = joblib.load(model_path)
        logger.info("Loading successful.")
    except Exception as e:
        logging_utilities.log_traceback(e, logger)
        raise


@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        result = model.predict(data)
        return json.dumps({"result": result.tolist()})
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

Overwriting score.py


In [123]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=env)

In [ ]:
  conda update -n base -c defaults conda

In [124]:
from azureml.core.webservice import LocalWebservice


# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration()

local_service = Model.deploy(ws, "test", [model], inference_config, deployment_config)

local_service.wait_for_deployment()

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry viennaglobal.azurecr.io
Logging into Docker registry viennaglobal.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM viennaglobal.azurecr.io/azureml/azureml_1b9697d50bbbb35eb098c299c7ed3dd0
 ---> e21868753f04
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> b2f323a4dd7b
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6Ijc4NGQ4NzQwLTRjNzAtNGQyMC04ZjQ1LWIyYmZlY2Y5NDAyOSIsInJlc291cmNlR3JvdXBOYW1lIjoibXlyZXNvdXJjZWdyb3VwIiwiYWNjb3VudE5hbWUiOiJ0b3JpYV93b3Jrc3BhY2UiLCJ3b3Jrc3BhY2VJZCI6IjU1ZDQ2Nzg0LTRkOTUtNGU0OC1hMjk1LWY4NDE3ZWU0YmIyZSJ9LCJtb2RlbHMiOnt9LCJtb2RlbHNJbmZvIjp7fX0= | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in b6696ece6795
 ---> cc21fa758cc5
Step 4/5 : RUN mv '/var/azureml-app/tmpwm64kx3g.py' /var/azureml-app/main.py
 ---> Running in f179437491ee
 ---> bfe7c6754dd6
Step 5/5 : CMD ["runsvdir","/va

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1,
                                               tags={"data": "loans",  "method" : "sklearn"},
                                               description='Predict loans with sklearn'
                                               )

In [111]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)

In [126]:

#%%time
#import uuid
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, 'best_remoterun_model')


#myEnv = Environment.get(workspace=ws, name="vfenv", version="1")
inference_config = InferenceConfig(entry_script="score.py", environment=env)

service_name = 'besttoberun' #+ str(uuid.uuid4())[:4]
service = Model.deploy(workspace=ws, 
                       name=service_name, 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [113]:
print(service.get_logs())

None


In [5]:
import requests
import json

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = 'http://6f524ec3-a118-48c8-9718-96ab0ed0a32d.eastus2.azurecontainer.io/score'
# If the service is authenticated, set the key or token
#key = ''

# Two sets of data to score, so we get two results back
data = {"data":
        [
          {
            "Column1": 10,
            "Experience": 10,
            "Income": 81,
            "Family": 3,
            "CCAvg": 0.6,
            "Education": 2,
            "Mortgage": 104,
            "Securities Account": 0,
            "CD Account": 0
          },
          {
            "Column1": 20,
            "Experience": 21,
            "Income": 193,
            "Family": 2,
            "CCAvg": 8.1,
            "Education": 3,
            "Mortgage": 0,
            "Securities Account": 0,
            "CD Account": 0
          },
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())




{"result": [0, 1]}
